> 提示：项目 Notebook 模板主要为了帮助同学了解如何划分报告结构，提示项目的注意事项。如果你理解项目的所有要求，那么完全可以自己创建新的 Notebook 来完成报告。

> 提交项目时，建议删除所有提示性文字，只保留你自己的 Markdown 说明文字和代码注释。

# 清理与分析数据（替换为你自己的报告标题）
## 收集

In [ ]:
# 导入需要的库


> 提示：你需要收集的文件有：
1. 收集手头文件 `twitter_archive_enhanced.csv`，其中包含了一些主要的推特信息，是本次清洗的主要数据，其中的评分、地位和名字等数据是从 `text` 原文中提取的，但是提取的并不好，评分并不都是正确的，狗的名字和地位也有不正确的 。**如果你想用评分、地位和名字进行分析和可视化，需要评估和清洗这些列。完成这些列的评估和清洗，你可以学到更加实用的技能。**
2. 编程下载收集互联网文件：`image-predictions.tsv`，其中包含了推特图像预测信息，根据推特中的图片预测出狗狗种类；
3. 查询 API 收集额外推特信息 `tweet_json.txt`，如果你无法访问 Twitter 的话，可以直接读取项目可供下载的 `tweet_json.txt` 文件，从中提取所需数据。至少需要提取转发数（`retweet_count`）和喜欢数（`favorite_count`）这两列，**如果你的分析中不需要用到其他列，则不需要收集其他列**。如果提取了其他列只用于清洗，那么这样的清洗没有意义。

In [ ]:
# 收集文件 1 保存为 dataframe


In [ ]:
# 收集文件 2 保存为 dataframe


In [ ]:
# 收集文件 3 保存为 dataframe


## 评估
### 目测评估

In [ ]:
# 目测评估三个数据集

In [ ]:
# 你需要添加更多的 code cell 和 markdown cell 来完成所有目测评估

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

### 编程评估

In [ ]:
# 使用 pandas 的各种方法评估三个数据集，比如 info value_counts 等

In [ ]:
# 你需要添加更多的 code cell 和 markdown cell 来完成所有编程评估

> 提示：
- 完成目测评估和编程评估之后，总结列出你发现的三个数据集中的所有问题；
- 每个问题都要有对应的一句话或几句话描述；
- 最终至少要包含 8 个质量问题和 2 个整洁度问题。

#### 质量
##### `dataframe_name1` 表格（将`dataframe_name1`修改为你的数据集名称，然后删掉括号内的提示）
- 问题的数量可能不同，你可以继续换行添加更多列表项，或者删除多余的 `- `
- 问题描述列表
- 问题描述列表
- 
- 
- 
- 
- 

##### `dataframe_name2` 表格（将`dataframe_name2`修改为你的数据集名称，然后删掉括号内的提示）
- 问题描述列表
- 问题描述列表
- 
- 


##### `dataframe_name3` 表格（将`dataframe_name3`修改为你的数据集名称，然后删掉括号内的提示）
- 问题描述列表
- 问题描述列表
- 
- 


#### 整洁度
- 问题描述列表
- 问题描述列表
- 



## 清理

> 提示：
- 清理数据集之前需要先备份数据集；
- 按照下面示例的结构：**定义-代码-测试**，对提出的每个问题进行清洗。

In [ ]:
# 备份三个数据集


#### 问题描述一

##### 定义
简述一下问题的解决方案

##### 代码

In [ ]:
# 解决问题一的代码


##### 测试

In [ ]:
# 测试问题一是否正确清理完成


#### 问题描述二

##### 定义
简述一下问题的解决方案

##### 代码

In [ ]:
# 解决问题二的代码


##### 测试

In [ ]:
# 测试问题二是否正确清理完成


In [ ]:
# 你需要添加更多的 code cell 和 markdown cell 来完成所有清洗


### 存储清理后的主数据集

In [ ]:
# 将清理后的主数据集保存为 csv 文件或者 SQLite 数据库文件


## 分析和可视化

> 提示：
- 在 Notebook 使用 pandas 或 SQL 分析主数据集，并生成至少三（3）个独立的结论。
- 在 Notebook 中使用 Python 绘图库或在 Tableau 中至少生成一（1）个可视化图表。


In [ ]:
# 分析或可视化代码


与上面分析或可视化相关的见解或说明

In [ ]:
# 分析或可视化代码


与上面分析或可视化相关的见解或说明

In [ ]:
# 你需要添加更多的 code cell 和 markdown cell 来完成所有分析和可视化


更多说明或总结等

> 提示：在完成 Notebook 的所有内容之后，还需要完成两篇文本和图片组成的 PDF 报告。因为这两篇报告中只是文字和图片，不需要包含代码，你可以使用文字编辑软件，比如 Word 来完成：
- 创建一个 300-600 字的书面报告，命名为 `wrangle_report.pdf`，在该报告中简要描述你的数据整理过程。这份报告可以看作是一份内部文档，供你的团队成员查看交流。
- 创建一个 250 字以上的书面报告，命名为 `act_report.pdf`，在该报告中，你可以与读者交流观点，展示你使用整理过的数据生成的可视化图表。这份报告可以看作是一份外部文档，如博客帖子或杂志文章。

> 提示：提交项目前建议删除 Notebook 中的所有提示性文字和注释，只保留自己的 Markdown 文本和代码注释。